In [21]:
%%capture
# update or install the necessary libraries
!pip install --upgrade openai
!pip install --upgrade python-dotenv

In [22]:
from pydantic import BaseModel
from openai import OpenAI

from dotenv import load_dotenv

load_dotenv()

# client session
client = OpenAI()

# Structured Outputs with OpenAI APIs

Reference blog: https://openai.com/index/introducing-structured-outputs-in-the-api/

Reference documentation: https://platform.openai.com/docs/guides/structured-outputs/introduction

## Simple Example of Structured Outputs

In [24]:
# Pydantic object
class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

# call the model
completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "Extract the event information."},
        {"role": "user", "content": "Alice and Bob are going to a science fair on Friday."},
    ],
    response_format=CalendarEvent,
)

event = completion.choices[0].message.parsed

In [25]:
event

CalendarEvent(name='Science Fair', date='Friday', participants=['Alice', 'Bob'])

## Structuring Chain-of-Thought Responses

In [29]:
class Step(BaseModel):
    explanation: str
    output: str

class MathReasoning(BaseModel):
    steps: list[Step]
    final_answer: str

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "You are a helpful math tutor. Guide the user through the solution step by step."},
        {"role": "user", "content": "how can I solve 8x + 7 = -23"}
    ],
    response_format=MathReasoning,
)

math_reasoning = completion.choices[0].message.parsed

In [28]:
math_reasoning

MathReasoning(steps=[Step(explanation='Subtract 7 from both sides to isolate the term with x.', output='8x + 7 - 7 = -23 - 7'), Step(explanation='Simplify both sides. On the left side, the +7 and -7 cancel out leaving just 8x. On the right side, we calculate -23 - 7.', output='8x = -30'), Step(explanation='Divide both sides by 8 to solve for x.', output='x = -30 / 8'), Step(explanation='Simplify the fraction by dividing the numerator and the denominator by 2.', output='x = -15 / 4')], final_answer='x = -\\frac{15}{4}')

## Structured Output with Function Calling

Useful when you are connecting the model to tools, functions, data, etc. in your system.

In [31]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_delivery_date",
            "description": "Get the delivery date for a customer's order. Call this whenever you need to know the delivery date, for example when a customer asks 'Where is my package'",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": {
                        "type": "string",
                        "description": "The customer's order ID.",
                    },
                },
                "required": ["order_id"],
                "additionalProperties": False,
            },
        },
        "strict": True, # enables structured outputs
    }
]

messages = []
messages.append({"role": "system", "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user."})
messages.append({"role": "user", "content": "Hi, can you tell me the delivery date for my order?"})

response = client.chat.completions.create(
    model='gpt-4o-2024-08-06',
    messages=messages,
    tools=tools,
)

In [32]:
response

ChatCompletion(id='chatcmpl-9uBILOlNgEqlyq1GehSbfDsXZCQ4V', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Could you please provide your order ID so I can look up the delivery date for you?', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1723176813, model='gpt-4o-2024-08-06', object='chat.completion', service_tier=None, system_fingerprint='fp_2a322c9ffc', usage=CompletionUsage(completion_tokens=19, prompt_tokens=107, total_tokens=126))

## Structured Output with `response_format`

Useful when the model needs to respond to the user in a specified structured way.

In [34]:
response = client.chat.completions.create(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "You are a helpful math tutor. Guide the user through the solution step by step."},
        {"role": "user", "content": "how can I solve 8x + 7 = -23"}
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "math_response",
            "schema": {
                "type": "object",
                "properties": {
                    "steps": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "explanation": {"type": "string"},
                                "output": {"type": "string"}
                            },
                            "required": ["explanation", "output"],
                            "additionalProperties": False
                        }
                    },
                    "final_answer": {"type": "string"}
                },
                "required": ["steps", "final_answer"],
                "additionalProperties": False
            },
            "strict": True
        }
    }
)

print(response.choices[0].message.content)

{"steps":[{"explanation":"We start with the given equation: 8x + 7 = -23.","output":"8x + 7 = -23"},{"explanation":"The goal is to isolate the variable 'x'. First, remove the constant term on the left side by subtracting 7 from both sides of the equation.","output":"8x + 7 - 7 = -23 - 7"},{"explanation":"Simplify both sides. The left side becomes 8x and the right side becomes -30, as 7 - 7 cancels out on the left side, and -23 - 7 equals -30 on the right side.","output":"8x = -30"},{"explanation":"Now, divide both sides by 8 to solve for x.","output":"8x / 8 = -30 / 8"},{"explanation":"The left side simplifies to x, and the right side simplifies to -3.75, because -30 divided by 8 is -3.75.","output":"x = -3.75"}],"final_answer":"x = -3.75"}
